#### Day 5: APIs

Instructions: 

Write your own Python script to answer the following questions: 
1. Which of these embassies is closest to the White House in meters? What is the address? 
2. If I wanted to hold a morning meeting there, which cafe would you suggest (best rating and closest)?
3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

Hint: 
- You will need to enable the `Google Places API`.
- You may find this page useful to learn about different findinging nearby places https://www.geeksforgeeks.org/python-fetch-nearest-hospital-locations-using-googlemaps-api/

In [1]:
pwd

'/Users/ysui/Desktop/PhD/MTE/pythoncamp2023_prep/Day05/Lab'

In [2]:
import importlib
import os
import googlemaps

os.chdir('/Users/ysui/Desktop/PhD/MTE/pythoncamp2023_prep/Day05/Lab')
imported_items = importlib.import_module('start_google')
gmaps = imported_items.client

whitehouse = '1600 Pennsylvania Avenue, Washington, DC'

embassies = [[38.917228,-77.0522365], 
	[38.9076502, -77.0370427], 
	[38.916944, -77.048739] ]


1. Which of these embassies is closest to the White House in meters? 

In [5]:
wh = gmaps.geocode(whitehouse)

def dist_finder(dest, origin = wh):
	orig_lat_long = origin[0]['geometry']['location']
	dist = gmaps.distance_matrix(orig_lat_long, dest)
	kms=[]
	for i in range(3):
		kms.append(float(dist['rows'][0]['elements'][i]['distance']['text'].split(' ')[0]))
	return dest[kms.index(min(kms))]

# find closest embassy
closest = dist_finder(dest = embassies)

closest

[38.9076502, -77.0370427]

What is the address? 

In [8]:
# address of closest
closest_address = gmaps.reverse_geocode(closest)[0]['formatted_address']
closest_address

'1601 Massachusetts Ave NW, Washington, DC 20036, USA'

2. If I wanted to hold a morning meeting there, which cafes would you suggest for breakfast (best rating and closest)?

In [9]:
bfast = gmaps.places_nearby(closest, type = 'cafe', rank_by = "distance", keyword = 'breakfast')['results']

print('Closest breakfast place is {} at {}'.format(bfast[0]['name'], bfast[0]['vicinity'])) # closest

# highest rated
rating = []
for i in range(len(bfast)):
	try:
		rating.append(bfast[i]['rating'])
	except:
		rating.append(0)
best_bfast = bfast[rating.index(max(rating))]

print('Best breakfast place is {} at {}'.format(best_bfast['name'], best_bfast['vicinity'])) # highest rated


Closest breakfast place is Galley Cafe at 1625 Massachusetts Ave NW, Washington
Best breakfast place is Earlybird Cafe at 1400 16th St NW b110, Washington


3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

In [10]:
bars = gmaps.places('bars near ' + '1617 Massachusetts Ave NW, Washington, DC 20036, USA')
dollar = []
for i in range(0, len(bars)):
	try:
	    dollar.append(bars['results'][i]['price_level'])
	except:
	    dollar.append(0)
fancy_bar = bars['results'][dollar.index(max(dollar))]

print('Fanciest bar is {} at {}'.format(fancy_bar['name'], fancy_bar['formatted_address'])) # fanciest

Fanciest bar is Public Bar Live at 1214 18th St NW, Washington, DC 20036, United States
